Implementation proposed by Alexandre Petkovic and Guillaume Delande  

The video presentation can be found at https://www.youtube.com/watch?v=xX_LhHFNmr4 

# Introduction

We propose an implementation of the paper "Generalized sorting with predictions" by Pinyan Lu, Xuandi Ren, Enze Sun, and Yubo Zhang (2020). The paper can be found at: https://arxiv.org/pdf/2011.00172.pdf. To this date, 11th of April 2021, no coded implementation has already been proposed.

Here we present how the code is divided and the constraints associated to using this code.
First, we generate a "true"/correct directed acyclic graph (DAG). Then, we run some code that will copy and modify the correct graph into a "kind-of-similar" graph, but with some mis-predicted edges. This is done in order to simulate a non-perfect prediction of the correct graph.

Later, we can then infer the correct graph from the predicted graph, by comparing edges. The main purpose of the paper is to obtain the sorted elements by minimizing the amount of probed edges. The code below achieves the correct sorting through a randomized algorithm which uses $O(n\log{n}+w)$ probings, where:  
$n$ is the number of correctly probed edges  
$w$ is the number of mispredicted edges

# Imports

In [1]:
import numpy as np
import random as rd
import copy

# Section 1: Generate true/correct and mispredicted graphs
In this section we will create a correct directed graph. In addition, we will simulate a prediction of that graph by copying the correct and modifying some of the edges, in order to create some mispredicted edges. This will be useful for testing the implementation of the algorithm.

Several functions are defined in the following cell:  
- ```generate_graph(n, pc)```: will create a list of edges, where each edge is stored in a tuple.
    - ```n```indicates the number of nodes that need to be contained in the graph.  
    - ```pc``` can be seen as a factor of randomness (used to create a binomial distribution of probability ```pc```), used to create some random edges/relationships between some of the nodes.
- ```create_mispredicted_graph```: will change the direction of some of the edges (generate mispredicted edges), based on a binomial experiment with probability ```ppred``` and a given graph (list of tuples). This is done by:
    - 1. looping through the list of edges
    - 2. For each edge (e.g. (a,b)), it performs a binomial experiment with probability ```ppred```
    - 3. If the binomial experiment succeeds it keeps (a, b), else it replaces it by (b, a).
   
- ```convert_tuples_to_dict```: converts a list of tuples into a Python dictionary data type. The above functions are executed on a list of edges (i.e. list of tuples). In the algorithm, it is required to have some arguments in dictionary data type, hence those lists of tuples need to be converted. The dictionary will have as key the vertex, and as value a list containing all the in_neighbors of that vertex/key. Hence, as key-pairs we have vertex-in_neighbors.   

Notes: 
- in the ```generate_graph``` function:  
    - we create a graph with successive nodes (i.e. use ```vertex = list(range(1,n+1))``` to generate the nodes). If one wants to create non-successive nodes (e.g. a graph with only nodes [1, 4, 749, 9580]) it has to be done manually.
    - always starts with node 1 (not 0!)  

- with the ```create_predicted_graph```, the only modifications that will be executed are to switch some of the edges in opposite direction of the correct direction. If one wants to add/remove new edges, it has to be done manually.

In [2]:
def generate_graph(n = 5, pc = 0.5): #Default values

    vertex = list(range(1,n+1)) # Generate a list of all the vertices
    edges = [] # create a list containing all the directed edges.

    # Since we ALWAYS start with node 1 (i.e. 1 cannot have an in-neighbor), we start the loop at 2.
    for keys in range(2,n+1):
        # Here we indicate between which elements the comparison is allowed between keys and those which are lower.
        # notice that we add the [1] at the end to indicate the fact that you can always compare keys and keys-1 (this
        # is required for you to have an Hamiltonian path).
        Connected = list(np.random.binomial(1, pc, size=(keys-2)))+[1]

        # Find the indices of all the 1 in the list of binomial numbers.
        indices = [i+1 for i, x in enumerate(Connected) if x == 1]

        # Now you generate all the edges starting with indices and ending with keys
        edges = edges + [(x, keys) for i, x in enumerate(indices)] #Generate a list of tuples. Tuples are less memory-intensive and faster

    return edges


def create_mispredicted_graph(correct_graph = generate_graph(), ppred = 0.7): #default
    PredictedEdges = []

    for indedges in correct_graph:
    # If the binomial experiment succeeds then we keep the vertex in the predicted. 
        randomexp = int( np.random.binomial(1, ppred, size=1) )
        if randomexp == 1:
            PredictedEdges = PredictedEdges + [indedges]
        else:
            PredictedEdges = PredictedEdges + [(indedges[1],indedges[0]) ] #tuple because faster and less memory-intensive

    return PredictedEdges


def convert_tuples_to_dict(list_of_edges): # This function takes the edges and puts them into a dictionary format

    if type(list_of_edges) is not list:
        return print("Your object is not a list ! Please try again")

    graph = {} #dict
    minimum = min(min(list_of_edges)) #returns the smallest node of the graph
    maximum = max(max(list_of_edges)) #returns the last/largest node of the graph

    for keys in range(minimum, maximum + 1): #+1 because range() doesn't include the last argument
        graph[keys] = [x[0] for i, x in enumerate(list_of_edges) if x[1]==keys]

    return graph

## Some examples:

In [34]:
#Default
print(generate_graph())
print(create_mispredicted_graph())
a = 1
convert_tuples_to_dict(a)

[(1, 2), (1, 3), (2, 3), (1, 4), (3, 4), (3, 5), (4, 5)]
[(2, 1), (3, 2), (1, 4), (2, 4), (3, 4), (1, 5), (4, 5)]
Your object is not a list ! Please try again


In [37]:
true_graph = generate_graph(n = 10, pc = 0.6)
mispredicted_graph = create_mispredicted_graph(correct_graph = true_graph, ppred = 0.7) #create a mispredicted graph based
print(f"true graph: {true_graph}")
print(f"mispredicted graph: {mispredicted_graph}")
convert_tuples_to_dict(mispredicted_graph) #return the graph in dictionary type

true graph: [(1, 2), (1, 3), (2, 3), (3, 4), (1, 5), (4, 5), (2, 6), (3, 6), (5, 6), (1, 7), (4, 7), (6, 7), (1, 8), (2, 8), (3, 8), (6, 8), (7, 8), (3, 9), (5, 9), (6, 9), (7, 9), (8, 9), (2, 10), (3, 10), (4, 10), (6, 10), (7, 10), (8, 10), (9, 10)]
mispredicted graph: [(2, 1), (1, 3), (2, 3), (3, 4), (1, 5), (5, 4), (6, 2), (3, 6), (5, 6), (1, 7), (7, 4), (7, 6), (8, 1), (8, 2), (3, 8), (6, 8), (8, 7), (3, 9), (5, 9), (6, 9), (9, 7), (8, 9), (10, 2), (3, 10), (4, 10), (10, 6), (7, 10), (8, 10), (10, 9)]


{1: [2, 8],
 2: [6, 8, 10],
 3: [1, 2],
 4: [3, 5, 7],
 5: [1],
 6: [3, 5, 7, 10],
 7: [1, 8, 9],
 8: [3, 6],
 9: [3, 5, 6, 8, 10],
 10: [3, 4, 7, 8]}

# Section 2: Create Relation matrix and path Discovery function
Later on, we will need to check if a path exists between two nodes of a (directed) graph. How that could be done is not assessed in the paper. We suggest two possibilities:
- one is to create a matrix that will keep track of the known relations between the nodes of the matrix. The matrix would then need to be updated when a new node is added.
- another one is to compute all possible paths in the graph starting from one of the nodes, and observing if at least one of those paths reaches the other node. This is more computationally intensive than the first solution, but relies on less assumptions.

We decide to opt for the matrix path exploration solution as it is simpler to implement and requires less computations. However, it relies on some assumptions, mentioned below.

Some notes about the matrix:  
- We will first start with an empty matrix, indicating that there are no known relations  
- The matrix will be of size (n,n), where n is the number of nodes in the graph
- For simplicity, we assume that the line numbers and column numbers of the matrix match the vertex' number. For instance, cell (1,2) of the matrix represents the edge (1,2) of the graph. Hence, cell (1,2) refers to the relation between vertex 1 and 2 (obviously 1<2). This has the advantage to facilitate the lookup in the matrix (see function Discovery). Note that already knowing the order of the nodes has no effect on the number of probes that need to be performed, which is the main goal of the paper (and code).  

In order to check for relations between nodes and identifying paths, we run the ```Discovery``` function over the matrix.
Some notes about the function ```Discovery```:  
- Because the lines and columns of the matrix match with the edges of the graph, we only need to compute the upper right half of the matrix. For instance, as we have the edge (1,2) mentioned in the upper right of the matrix and the edge in opposite direction (2,1) is wrong (because 1<2) and does not bring any additional information, it is not required to represent the lower left half of the matrix. In addition, an edge will always be probed before adding it to the matrix. Therefore, the new edge to add in the matrix will always be in the right direction (because of the ```probe``` function that will be explained later.  
- the function takes two inputs:  
    - 1. The state of the currently known relations between vertices, i.e. the relation matrix  
    - 2. The latest edge which was probed. The input here must be the correct edge orientation. That is, the inputs are expected to be of the type (a,b) with a<b.  
- an edge going from an incoming neighbor to a vertex is represented by -1 in the matrix.
- Assumes that we start with node 1 (not 0).

In [3]:
def Discovery(relation_matrix, new_edge):
    
    #relation_matrix is the matrix highlighting the relations between the different nodes/vertices of the graph
    # new edge is the relation between two vertices that needs to be added to the matrix
    
    if new_edge[1] < new_edge[0]: #should not happen if you probe() before calling Discovery
        print("You have submitted an edge where the in_neighbor > outgoing vertex. This is not possible.")
        return relation_matrix
    
    # We now assume Edge[0] < Edge[1])
    relation_matrix[ new_edge[0]-1, new_edge[1]-1 ] = -1 #set to -1 to indicate the edge going from in_neighbor to outgoing vertex
    #substract -1 in the matrix rows and columns to match with the vertices' numbers, that start at 1 and not 0.
    
    """Now that we have set matrix(Edge[0], Edge[1]) to -1, indicating that Edge[0]<Edge[1], we need to compute all the
    remaining order relations which we can deduce from that one."""
        
    """CASE 1: all elements lower than Edge[0] will be lower than Edge[1]
    In practice, if we know that Edge' < new_edge[0] then we also know, by transitivity, that Edge' < new_edge[1]. 
    Now, we can find all edges which are CURRENTLY known to be lower than Edge[0]. 
    This is done by identifying the rows located in:
    column new_edge[0] and between row 1 and new_edge[0]-1 with an index equal to -1.
    """
    try:
        lower = [i for i, x in enumerate(relation_matrix[0:new_edge[0]-1, new_edge[0]-1]) if x == -1]
        relation_matrix[lower, new_edge[1]-1] = -1
    except:
        pass

    """CASE 2: all elements larger than new_edge[1] will also be larger than new_edge[0] and those smaller than it."""
    try:
        # Find all elements which are lower than new_edge[1]
        lowerup = [i for i, x in enumerate(relation_matrix[0:new_edge[1]-1, new_edge[1]-1 ]) if x == -1]
        # Find all elements which are larger than new_edge[1]
        higher = [(i + new_edge[1]) for i, x in enumerate(relation_matrix[(new_edge[1]-1), new_edge[1]:n]) if x == -1]
        # Use an if elif condition. We use this because we effectively need to loop over the list higher and this can only
        # occur if the length of that one is larger than 1.
        if len(higher)== 1:
            relation_matrix[lowerup, higher] = -1
        elif len(higher)> 1:
            for vertex in higher:
                vertex = vertex
                relation_matrix[lowerup, vertex] = -1
    except:
        pass
    
    return relation_matrix

## Some examples
Let's show the intuition behind the ```Discovery```function through an example. The idea is that, as we probe more edges, we add more edges to the matrix. In addition, the function will compute whether there is an existing path between two nodes by checking for transisitivity (i.e. if a is connected to b, and b is connected to c, then there is a path from a to c).  

The row number correspond to the in-neighbor and the column to the vertex. For instance, the edge (1,2) will be indicated by a -1 at position: [row=1, col=2] in the matrix.

In [46]:
n = 4 #number of vertices
Relation = np.zeros((n, n), dtype=int) #n vertex in the graph

Edge = (1,2)
Relation = Discovery(Relation, Edge) #add a first edge to the matrix
print(f"added {Edge}\n{Relation}\n")

Edge = (3,4)
Relation = Discovery(Relation, Edge) #add second edge, 
#note: we already knew the edge (2,3), hence there's is a directed path from 1 to 3 (transitivity), 
#which is indicated by -1 at position [row=1,col=3] in the matrix
print(f"added {Edge}\n{Relation}\n")

Edge = (2,3)
Relation = Discovery(Relation, Edge)
print(f"added {Edge}\n{Relation}\n")


added (1, 2)
[[ 0 -1  0  0]
 [ 0  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0  0]]

added (3, 4)
[[ 0 -1  0  0]
 [ 0  0  0  0]
 [ 0  0  0 -1]
 [ 0  0  0  0]]

added (2, 3)
[[ 0 -1 -1 -1]
 [ 0  0 -1 -1]
 [ 0  0  0 -1]
 [ 0  0  0  0]]



As we've progressively probed edges and added them to the matrix, we can see which vertices are connected by a directed path. This is the intuition used in our implementation of the algorithm.

# Section 3: The Algorithm

Please refer to the paper for more detailed explanations. We provide only the most important explanations in the code below. Here, we present the algorithm assuming that S_u is unknown and the algorithm needs to infer it.

In [3]:
class Algorithm_unknown_Su(object):
    
    def __init__(self, predicted_graph = convert_tuples_to_dict(create_mispredicted_graph(correct_graph = generate_graph()))):
    #the default values are the ones from the aforementioned graph functions
        # we expect the input arguments to be dictionaries
        if (type(predicted_graph) is not dict):
            return print(f"one of the arguments is not in dict type")
        
        self.predicted_graph = predicted_graph
        

        self.list_of_all_vertices = self.predicted_graph.keys()
        self.T_u = self.predicted_graph #at the start, T_u = predicted in_neighbors of u
        #T_u corresponds to the in_neighbors of u that are not known to be uncorrect
        
        self.certif_1 = {} #need to keep track of the certificates
        self.certif_2 = {}
        
        self.A_set = set() # A starts as empty set. (It is easier to deal with if it is a set)
        self.V_set = set(self.list_of_all_vertices) 
        
        self.n = len(self.V_set) #n is the number of vertices in the graph, i.e. number of vertex we will have to connect with each other
        self.A_matrix = np.zeros((self.n, self.n), dtype=int) # n is the number of vertex.
        
        self.relation_matrix = np.zeros((self.n, self.n), dtype=int) #keep track of probed edges (and to compute paths)


####################### FUNCTIONS TO USE LATER

    def probe(self, in_neighbor, vertex): # the function returns the correct Edge
        if in_neighbor < vertex:
            return (in_neighbor, vertex)
        else:
            return (vertex, in_neighbor)

    def update_T_u(self, new_Q):  #takes in a list of tuples
        """The goal of this function is to update T_u
        As we probe more edges, we get more information about the correct edges, hence we can also infer 
        the incorrect edges (e.g. if we know that (a,b) is correct, then we also know (b,a) is incorrect)
        Therefore, the goal is to iteratively remove more and more elements from T_u

        The idea is that each time a new edge is added to Q (i.e. considered as correct for sure), 
        we remove the eventual incorrect edges from T_u. This prevents us to recompute the whole T_u every time
        """
        #assuming the argument new_Q is a list of new edges (in tuple type)
        if (type(new_Q) is not list):
            return print(f"one of the arguments is not in dict type")
  
        for each_tuple in new_Q:
            (in_neighbor, vertex) = each_tuple #extract the info from the tuple

            #no need to probe, we assume it is already done before calling this function

        ### SANITY CHECKS
            if vertex not in self.T_u: #check that vertex is indeed a node of T_u (sanity check because it should always be!)
                self.T_u[vertex] = []  #create list to store in_neighbors

            if in_neighbor not in self.T_u[vertex]: #SANITY CHECK, because should always be in T_u unless we've removed it
                self.T_u[vertex].append(in_neighbor) 

        ### REMOVE opposite direction (i.e. the tuple (vertex, in_neighbor)) of probed edge from T_u   
                # if (v,u) is in new_Q (meaning it is added to Q), then (u,v) should be removed from T_u
            if (in_neighbor in self.T_u) and (vertex in self.T_u[in_neighbor]): #if key exists AND is equal to 
                self.T_u[in_neighbor].remove(vertex) #remove the value (vertice) from that in_neighbor

        return self.T_u
    

    def check_condition_1(self, u, A): #assuming A is a set
        """returns False if condition 1 for u being an ideal vertex is satisfied
        and True if condition 1 is not satisfied."""
        
        if set(self.T_u[u]).issubset(A) == True:
            return [False]
        else:
            return [True, set(self.T_u[u]) - A]
        
        
    def check_condition_2(self, u, Relation_matrix):
        """returns True if condition 2 for u being an ideal vertex is NOT satisfied.
        Returns False if condition 2 for u being an ideal vertex IS satisfied
        """
        condition_2_candidate = set() 
        #We will store the key u that validates the condition, and as value a list of probed edges (that have made u a valid condition)

        if len(self.T_u[u]) <= 1:
            return [False] #False = condition 2 cannot be satisfied

        else: #i.e. there are at least 2 in_neighbours for u in T_u
            for v in self.T_u[u]:
                for w in self.T_u[u]: #Not optimal because we go over all the possible edges twice!
                    if v != w:

                        Edge = self.probe(in_neighbor = v, vertex = w) #
                        
                        if Relation_matrix[Edge[0]-1,Edge[1]-1] == 0: # meaning that we do not know the order between the two elements.
                            # (no path between v1 and v2) AND (v1 and v2 are in A)

                            condition_2_candidate.add(Edge[0])
                            condition_2_candidate.add(Edge[1])
                            
            #if len(condition_2_candidate.keys()) > 0:
            if len(condition_2_candidate) > 0:
                return [True, condition_2_candidate] 

            return [False] 


    def equalSu(self, u, Relation):
        """Purpose: compare T_u with S_u.
        As we don't know the true graph, we have to infer S_u. 
        This can be done by taking T_u and checking if all elements in T_u have already been probed or infered (i.e. are correct).
        Only then, T_u will be eaqual to S_u and we will be able to check for certificates. 
        This function will be used to know if we can assign/check certificates or not by checking probed edges in the Relation matrix.
        Returns True if T_u = S_u, and False if not all elements of T_u have been probed yet."""

        if len(self.T_u[u]) == 0:
            return True

        elif len(self.T_u[u]) == 1:
            #coordinate = probe(T_u[u][0], u)
            coordinate = (self.T_u[u][0], u)

            if Relation[coordinate[0]-1,coordinate[1]-1] == 0: #has not been added to the probed elements (matrix)
                return False
            else:
                return True

        elif len(self.T_u[u]) > 1:
            for lower in self.T_u[u]:
                #coordinate = probe(in_neighbor = lower, vertex = u)
                coordinate = (lower, u)
                # If the matrix entry equals 0 it means that this edge has not been probed nor deduced.
                if Relation[coordinate[0]-1, coordinate[1]-1] == 0:
                    return False

            return True

    # The last possibility being that you are dealing with an element such that its in neighboor is empty.
        else:
            return True
        
        
    def Discovery(self, relation_matrix, new_edge):
        # relation_matrix is the matrix highlighting the relations between the different nodes/vertices of the graph
        # new edge is the relation between two vertices that needs to be added to the matrix

        if new_edge[1] < new_edge[0]: #should not happen if you probe() before calling Discovery
            print("You have submitted an edge where the in_neighbor > outgoing vertex. This is not possible.")
            return relation_matrix

        # We now assume Edge[0] < Edge[1])
        relation_matrix[ new_edge[0]-1, new_edge[1]-1 ] = -1 #set to -1 to indicate the edge going from in_neighbor to outgoing vertex
        #substract -1 in the matrix rows and columns to match with the vertices' numbers, that start at 1 and not 0.

        """Now that we have set matrix(Edge[0], Edge[1]) to -1, indicating that Edge[0]<Edge[1], we need to compute all the
        remaining order relations which we can deduce from that one."""

        """CASE 1: all elements lower than Edge[0] will be lower than Edge[1]
        In practice, if we know that Edge' < new_edge[0] then we also know, by transitivity, that Edge' < new_edge[1]. 
        Now, we can find all edges which are CURRENTLY known to be lower than Edge[0]. 
        This is done by identifying the rows located in:
        column new_edge[0] and between row 1 and new_edge[0]-1 with an index equal to -1.
        """
        try:
            lower = [i for i, x in enumerate(relation_matrix[0:new_edge[0]-1, new_edge[0]-1]) if x == -1]
            relation_matrix[lower, new_edge[1]-1] = -1
        except:
            pass

        """CASE 2: all elements larger than new_edge[1] will also be larger than new_edge[0] and those smaller than it."""
        try:
            # Find all elements which are lower than new_edge[1]
            lowerup = [i for i, x in enumerate(relation_matrix[0:new_edge[1]-1, new_edge[1]-1 ]) if x == -1]
            # Find all elements which are larger than new_edge[1]
            higher = [(i + new_edge[1]) for i, x in enumerate(relation_matrix[(new_edge[1]-1), new_edge[1]:n]) if x == -1]
            # Use an if elif condition. We use this because we effectively need to loop over the list higher and this can only
            # occur if the length of that one is larger than 1.
            if len(higher)== 1:
                relation_matrix[lowerup, higher] = -1
            elif len(higher)> 1:
                for vertex in higher:
                    vertex = vertex
                    relation_matrix[lowerup, vertex] = -1
        except:
            pass

        return relation_matrix    
    
#############################
#############################
#############################

    def run_algo_Su_unknown(self):
        
        # Counts and stuff
        mis_pred = 0
        correct_pred = 0
        probed = 0
        
        probed_correct = 0
        probed_wrong = 0
        
        while len(self.V_set - self.A_set) != 0:
            print(f"##########")        
        
####### CHECK IF CERTIFICATES ARE STILL VALID ########
#Since we go to the next iteration, we need to check if the certificates of the previous iteration still hold
# No need to check certificates for all vertices, we only check for the certificates of the smallest value that is not in A yet.
        
            # CHECK CERTIFICATE 1
            if len(self.certif_1.keys()) >= 1:
                element = min(self.certif_1.keys()) #No need to check for all vertices, the smallest one should end up being active...
                if (self.check_condition_1(u = element, A = self.A_set)[0] == False) and (self.equalSu(u = element, Relation = self.relation_matrix) == True): #no certificate of type 1 for the key 'element' is found
                    print(f"certificate 1 of vertex u = {element} is not valid anymore")
                    del self.certif_1[element] #then we remove key 'element' from the certif_1 dict (that contains only keys that have certif 1)
                    print(f"removed the certificate 1 from: {element}")


            # CHECK CERTIFICATE 2
            if len(self.certif_2.keys()) >= 1:
                element = min(self.certif_2.keys()) #No need to check for all vertices, the smallest one should end up being active...
                if (self.check_condition_2(u = element, Relation_matrix = self.A_matrix)[0] == False) and (self.equalSu(u = element, Relation = self.relation_matrix) == True): #no certificate of type 1 for the key 'element' is found
                    print(f"certificate 2 of vertex u = {element} is not valid anymore")
                    del self.certif_2[element] #then we remove key 'element' from the certif_1 dict (that contains only keys that have certif 1)
                    print(f"removed the certificate 2 from: {element}")

####### FIND U ########

            # LIST CANDIDATES FOR U
            candidate = self.V_set - self.A_set - set(self.certif_1.keys()) - set(self.certif_2.keys())
            print(f"list of candidates: {candidate}")
            #For understanding what is happening, why some vertices are not candidates
            print(f"The vertices that contain a valid certificate 1 are : {self.certif_1.keys()}") 
            print(f"The vertices that contain a valid certificate 2 are : {self.certif_2.keys()}")

            # SELECT U
            # For making sure the code is properly developed, we choose u randomly BUT, once running properly it is better
            # to choose the smallest candidate because for example, if one mispredicted edge (e.g. (2,1)) is not probed BEFORE
            # randomly chosing u = 2, then we will have no in-neighbors for 2 and hence 2 might be added to A while vertex 1 is not added yet.
            # Taking the minimum ensures that at least the edges of the smallest vertex are correct as they are being probed for sure.
            
            #u = rd.sample(candidate, k=1)[0] #Take a random value for u
            u = min(candidate) #take the smallest index of the available ones.
            print(f"chosen u is: {u}")
        
####### VERIFY CONDITIONS OF IDEAL VERTEX ########
        # We want to verify if the the two conditions for being an IDEAL vertex are satisfied. 
        # If they are satisfied, then the 'if' and 'elif' conditions will NOT be run and the 'else' will be effective
        
            condit_1 = self.check_condition_1(u = u, A = self.A_set) #if the first element returns True, then u DOES NOT satisfy condition 1
            condit_2 = self.check_condition_2(u = u, Relation_matrix = self.A_matrix)
            print(f"vertices that prevent u={u} from satisfying condition 1: {condit_1}")
            print(f"vertices that prevent u={u} from satisfying condition 2: {condit_2}")

    ######## CONDITION 1 NOT SATISFIED
            if condit_1[0] == True: # Meaning that condition 1: IS NOT satisfied (and a certificate will be assigned to u iff T_u = S_u)

                # FIND AN IN-NEIGHBOR V FOR U
                print("Condition 1 for u being an ideal vertex is NOT satisfied.")
                candidate = set(condit_1[1]) # Find candidate in-neighbors v for vertex u
                v = rd.sample(candidate, k=1)[0] #Randomly select v amongst the candidates
                print(f"Candidates for v are: {candidate}\nwe randomly pick v: {v}")

                # PROBE (V,U)
                true_edge = self.probe(in_neighbor = v, vertex = u) #PROBE
                
                # Update probe
                probed += 1
                if true_edge != (v,u):
                    print(f"in-neighbor v = {v} is larger than {u}, hence is a mis-predicted edge whose direction is now being reversed.")
                    mis_pred += 1

                else: 
                    print(f"The edge {true_edge} is probed and correct.")
                    correct_pred += 1
                    
                
                # UPDATES
                self.update_T_u(new_Q = [true_edge]) # add the correct edge to T_u and remove the opposite direction of the correct edge from T_u
                self.relation_matrix = self.Discovery(relation_matrix = self.relation_matrix, new_edge = true_edge) #Record the edge in the matrix of probed edges 

                # CHECK IF A CERTIFICATE TYPE 1 IS APPLICABLE (i.e. if T_u = S_u, which is if all vertices in T_u have already been probed)
                if self.equalSu(u, Relation = self.relation_matrix) == True: # IF True, then T_u = S_u --> you do have a certificate of type 1.        

                    if true_edge == (v,u):
                        self.certif_1[u] = v #list the in-neighbors v that caused u to have a valid certificate of type 1
                        print(f" u: {u} has a valid certificate 1, due to (at least) v: {v}")
                else: 
                    print(f"All in-neighbors of vertex u = {u} (contained in T_u[{u}]) have not been probed yet.")
                    print(f"Therefore, we can not give a certificate 1, as we are not sure T_u[{u}] equals S_u[{u}]")


    ######## CONDITION 2 NOT SATISFIED
            elif condit_2[0] == True: # if True, condition 2 for being an ideal vertex is NOT satisfied
                print("Condition 2 for u being an ideal vertex is NOT satisfied.")

                # FIND 2 IN-NEIGHBORS V1 AND V2 FOR U
                v = rd.sample(condit_2[1],k=2) #randomly select a pair
                print(f"sampled two points: {v[0]} and {v[1]}")

            ##### PROBE (V1,U) and (V2,U)
                
                # PROBE (V1, U)

                True_Edge_1 = self.probe(in_neighbor = v[0], vertex = u) #Need to probe
                probed += 1

                if True_Edge_1 == (v[0], u):
                    correct_pred += 1
                else:
                    mis_pred += 1
                
                print(f"true edge 1 is: {True_Edge_1}")                
                
                #PROBE (V2, U)
                True_Edge_2 = self.probe(in_neighbor = v[1], vertex = u) #Need to probe
                probed += 1

                if True_Edge_2 == (v[1], u):
                    correct_pred += 1
                else:
                    mis_pred += 1

                print(f"True edge 2 is: {True_Edge_2}")      
                

                # UPDATES
                self.update_T_u(new_Q = [True_Edge_1, True_Edge_2])
                self.relation_matrix = self.Discovery(self.relation_matrix, new_edge = True_Edge_1)
                self.relation_matrix = self.Discovery(self.relation_matrix, new_edge = True_Edge_2)

                # CHECK IF A CERTIFICATE TYPE 2 IS APPLICABLE (i.e. if T_u = S_u, which is if all vertices in T_u have already been probed)
                if (self.equalSu(u, Relation = self.relation_matrix) == True) and (True_Edge_1 == (v[0],u)) and (True_Edge_2 == (v[1],u)):
                    self.certif_2[u] = (v[0],v[1])
                    print(f" u: {u} has a valid certificate 2, due to (at least) v1 = {v[0]} and v2 = {v[1]}.")


    ######## CONDITION 1 AND 2 SATISFIED: U IS AN IDEAL VERTEX
            else:
                print("conditions for u to be an ideal vertex are met")

                # Create while loop to find largest t (in T_u), that is still smaller than u (i.e. (t,u) is the right direction)
                verif = True #will be used in the next while loop
                to_check = list(set(self.T_u[u]).intersection(self.A_set)) #list of in-neighbors that are already in A (i.e. in the order A)
                print(f"T_u for u is: {to_check}")

                while verif == True: #need to find the max of T_u[u], that is still smaller than u
                    if len(to_check) > 0 :
                        
                        t = max(to_check) #Need to check
                        
                        #t needs to be accessible in A_matrix
                        
                        true_edge = self.probe(in_neighbor = t, vertex = u) #Need to probe
                        probed += 1
                        if true_edge == (t, u):
                            correct_pred += 1
                        else:
                            mis_pred += 1
                            
                        #Since we probe, we need to update. Whatever the result of the probe.
                        print(f"max t is {t} and true edge: {true_edge}")

                        if true_edge == (t, u): #i.e. t < u, we can end the loop because we have the right thing
                            print(f"the edge is correct, we can add u to A")
                            
                            #ALL in-neighbors in T_u are correct by default, as they are smaller than t
                            for i in to_check:
                                edge = (i,u)
                                self.relation_matrix = self.Discovery(relation_matrix = self.relation_matrix, new_edge = edge)
                                self.update_T_u(new_Q = [edge]) #update T_u, such that if there is an edge with opposite direction in the graph, it is discarded
                            verif = False

                        elif true_edge == (u,t) and len(to_check)>1: #Problem: t > u, so we have to look for the next highest t until t < u
                            #there is a next highest t
                            
                            print(f"the probe found a mispredicted max t: {true_edge} is a mispredicted edge and we look for the next highest t.")
                            to_check.remove(t) #so that it can restart the loop and take the next highest t
                            self.update_T_u(new_Q = [true_edge]) #Need to make sure it is updated, especially if t > u !
                            self.relation_matrix = self.Discovery(relation_matrix = self.relation_matrix, new_edge = true_edge) #update matrix of probed edges

                        else: #should not happen
                            print(f"problem, the edge is not in the correct direction, but we don't have another highest t")
                            print(f"to_check: {to_check}")

                    else: # only if len(Tocheck) == 0: and True_Edge == (u,t)
                        true_edge = (u,u) #won't happen unless T_u is empty (like for 1:[] ...)
                        #correct_pred += 1 #we don't count them as a probe
                        #probed += 1
                        verif = False # there is nothing else to do.

        
        #####UPDATES: ADD U TO A, RECORD PROBES IN THE MATRIX, UPDATE T_U        
                
                self.A_set.add(u) #add u to A
                print(f"{u} was added to A")

                self.A_matrix = self.Discovery(relation_matrix = self.A_matrix, new_edge = true_edge) #update matrix A
                #if there is an edge in A that can go from t to u
                
                self.relation_matrix = self.Discovery(relation_matrix = self.relation_matrix, new_edge = true_edge) #update matrix of probed edges
                for in_neighbor in self.T_u[u]: #by default all the other in-neighbors are smaller than t, therefore we know they are in the correct direction (because t < u)
                    correct_edge = (in_neighbor, u)
                    self.relation_matrix = self.Discovery(relation_matrix = self.relation_matrix, new_edge = correct_edge)
                self.update_T_u(new_Q = [true_edge]) #update T_u
                print("updates are done")

            print(f"A_set is now: {self.A_set}")
            print(f"mis-pred: {mis_pred}\ncorrect_pred:{correct_pred}")
        
        print("##########################\n###### FINAL RESULT")
        print(f"Sorting successful, A is : {self.A_set}") #At the very end, out of the 'While' loop
        print(f"mis-predicted edges (w): {mis_pred}")
        print(f"correct probes: {correct_pred}")
        print(f"probed edges: {probed}")
        
        return self.A_set, self.T_u

# Section 4: Final example

First, let's use the functions we have presented at the beginning to create a true/correct graph, and a graph with mispredicted edges ,derived by the correct graph.

In [4]:
# Please do not forget to run the 'import' cells as well as the ones to generate graphs (see section 1), 
# and the algorithm class (section 3).

# Create graphs "randomly"
true_graph = generate_graph(n = 20, pc = 0.6)
mispredicted_graph = create_mispredicted_graph(correct_graph = true_graph, ppred = 0.7) #create a mispredicted graph based

#Show the created graphs
#print(f"true graph: {true_graph}")
print(f"mispredicted graph: {mispredicted_graph}")
print(f"number of edges in the graph: {len(mispredicted_graph)}")

# Turn them into dictionaries, to feed into the algorithm
mispredicted_graph_dict = convert_tuples_to_dict(mispredicted_graph)

#import copy
#Need to make a copy of the dict, otherwise the modifications will be applied to the original dict 
dict_1 = copy.deepcopy(mispredicted_graph_dict)

mispredicted graph: [(2, 1), (3, 1), (3, 2), (1, 4), (3, 4), (1, 5), (5, 2), (3, 5), (4, 5), (2, 6), (5, 6), (1, 7), (7, 2), (3, 7), (7, 5), (6, 7), (2, 8), (3, 8), (4, 8), (8, 6), (7, 8), (1, 9), (3, 9), (5, 9), (6, 9), (7, 9), (8, 9), (1, 10), (10, 2), (10, 6), (7, 10), (10, 8), (9, 10), (11, 1), (11, 4), (11, 5), (11, 8), (9, 11), (10, 11), (12, 1), (12, 3), (4, 12), (12, 5), (12, 6), (9, 12), (12, 11), (13, 2), (13, 3), (13, 5), (6, 13), (7, 13), (13, 8), (10, 13), (11, 13), (12, 13), (1, 14), (2, 14), (3, 14), (4, 14), (5, 14), (6, 14), (7, 14), (8, 14), (14, 9), (14, 10), (14, 12), (14, 13), (1, 15), (3, 15), (4, 15), (5, 15), (15, 6), (7, 15), (15, 8), (15, 11), (12, 15), (13, 15), (14, 15), (16, 2), (3, 16), (4, 16), (5, 16), (6, 16), (7, 16), (8, 16), (11, 16), (13, 16), (16, 14), (15, 16), (17, 4), (5, 17), (17, 6), (17, 8), (9, 17), (10, 17), (17, 11), (16, 17), (1, 18), (2, 18), (18, 3), (5, 18), (7, 18), (18, 8), (18, 10), (18, 11), (18, 13), (14, 18), (18, 15), (18, 17), 

In [5]:
dict_1

{1: [2, 3, 11, 12],
 2: [3, 5, 7, 10, 13, 16, 19],
 3: [12, 13, 18],
 4: [1, 3, 11, 17],
 5: [1, 3, 4, 7, 11, 12, 13],
 6: [2, 5, 8, 10, 12, 15, 17],
 7: [1, 3, 6],
 8: [2, 3, 4, 7, 10, 11, 13, 15, 17, 18, 19, 20],
 9: [1, 3, 5, 6, 7, 8, 14],
 10: [1, 7, 9, 14, 18],
 11: [9, 10, 12, 15, 17, 18, 19, 20],
 12: [4, 9, 14],
 13: [6, 7, 10, 11, 12, 14, 18],
 14: [1, 2, 3, 4, 5, 6, 7, 8, 16, 19],
 15: [1, 3, 4, 5, 7, 12, 13, 14, 18, 19],
 16: [3, 4, 5, 6, 7, 8, 11, 13, 15],
 17: [5, 9, 10, 16, 18],
 18: [1, 2, 5, 7, 14, 19, 20],
 19: [3, 10, 16, 17],
 20: [1, 4, 7, 9, 10, 15, 19]}

In [6]:
a = Algorithm_unknown_Su(predicted_graph = dict_1)
a.run_algo_Su_unknown()

##########
list of candidates: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
The vertices that contain a valid certificate 1 are : dict_keys([])
The vertices that contain a valid certificate 2 are : dict_keys([])
chosen u is: 1
vertices that prevent u=1 from satisfying condition 1: [True, {11, 2, 3, 12}]
vertices that prevent u=1 from satisfying condition 2: [True, {11, 2, 3, 12}]
Condition 1 for u being an ideal vertex is NOT satisfied.
Candidates for v are: {11, 2, 3, 12}
we randomly pick v: 3
in-neighbor v = 3 is larger than 1, hence is a mis-predicted edge whose direction is now being reversed.
All in-neighbors of vertex u = 1 (contained in T_u[1]) have not been probed yet.
Therefore, we can not give a certificate 1, as we are not sure T_u[1] equals S_u[1]
A_set is now: set()
mis-pred: 1
correct_pred:0
##########
list of candidates: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
The vertices that contain a valid certificate 1 are :

({1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20},
 {1: [],
  2: [1],
  3: [1, 2],
  4: [1, 3],
  5: [1, 3, 4, 2],
  6: [2, 5],
  7: [1, 3, 6, 2, 5],
  8: [2, 3, 4, 7, 6],
  9: [1, 3, 5, 6, 7, 8],
  10: [1, 7, 9, 2, 6, 8],
  11: [9, 10, 1, 4, 5, 8],
  12: [4, 9, 1, 3, 5, 6, 11],
  13: [6, 7, 10, 11, 12, 2, 3, 5, 8],
  14: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13],
  15: [1, 3, 4, 5, 7, 12, 13, 14, 6, 8, 11],
  16: [3, 4, 5, 6, 7, 8, 11, 13, 15, 2, 14],
  17: [5, 9, 10, 16, 4, 6, 8, 11],
  18: [1, 2, 5, 7, 14, 3, 8, 10, 11, 13, 15, 17],
  19: [3, 10, 16, 17, 2, 8, 11, 14, 15, 18],
  20: [1, 4, 7, 9, 10, 15, 19, 8, 11, 18]})

# Section 5: Alternative implementation
Here we propose an idea to implement the algorithm somewhat differently, this is more computationally intensive but also more suited for larger graphs, where a matrix would become too "heavy", and where the vertices are not assumed to be in a successive order.  
We have based our implementation of the algorithm on the idea of having a matrix keeping track of the different edges that were added to A and easily computes/identifies a path between two different vertices. However, this approach has some drawbacks as well. Hence, another approach could consider having a function run over A (that would be under the form of a dictionary) and check all paths starting from a specific vertex, adding the paths to a list of all possible paths. If the two vertices are found to be in at least one same path (iterate over that list, and check if both are mentioned in a same path), we would know there is a partial order in A for those two vertices (they're comparable by transitivity). Hence, we can also use this technique to check for a (non)valid certificate 2 and a (non)valid condition 2 for ideal vertices. One such function could be of the form below and should be run once for each direction, i.e. if we wanted to check if there is a partial order between a and b (in A), we would have to run once with ```start_vertex = a``` and once with ```start_vertex = b``` in order to take both directions into consideration.

In [77]:
# the code below was retrieved from: https://www.python-course.eu/graphs_python.php

def find_all_paths(self, start_vertex, end_vertex, path=[]):
    """ find all paths from start_vertex to 
        end_vertex in graph """
    graph = self.__graph_dict 
    path = path + [start_vertex]
    if start_vertex == end_vertex:
        return [path]
    if start_vertex not in graph:
        return []
    paths = []
    for vertex in graph[start_vertex]:
        if vertex not in path:
            extended_paths = self.find_all_paths(vertex, 
                                                 end_vertex, 
                                                 path)
            for p in extended_paths: 
                paths.append(p)
    return paths